In [3]:
import torch
from torch import nn
import sys
import json
import os
import multiprocessing

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")
from d2lzh_pytorch.BERT_unit_v3.use_BERT import load_pretrained_model, \
SNLIBERTDataset, BERTClassifier


from d2lzh_pytorch.myUtils import try_all_gpus

In [4]:
devices = try_all_gpus()
# devices = [torch.device('cpu')]

# 加载词向量
bert, vocab = load_pretrained_model('bert.small',
                                    num_hiddens=256,
                                    ffn_num_hiddens=512,
                                    num_heads=4,
                                    num_layers=2,
                                    dropout=0.1,
                                    max_len=512,
                                    devices=devices)

In [11]:
# 如果出现显存不足错误，请减少“batch_size”。在原始的BERT模型中，max_len=512
batch_size = 512
max_len = 128
num_workers = 4

# data_dir = download_extract('SNLI')

data_dir = r"/home/mylady/code/python/DL-pytorch/apps/chapter_pytorch_demo/data/snli_1.0"

In [12]:

train_set = SNLIBERTDataset(read_snli(data_dir, True), max_len, vocab)
test_set = SNLIBERTDataset(read_snli(data_dir, False), max_len, vocab)

read 549367 examples
read 9824 examples


In [8]:
# 加载数据
train_iter = torch.utils.data.DataLoader(train_set,
                                         batch_size,
                                         shuffle=True,
                                         num_workers=num_workers)

test_iter = torch.utils.data.DataLoader(test_set,
                                        batch_size,
                                        num_workers=num_workers)

In [9]:
# 模型及参数
net = BERTClassifier(bert)
lr = 1e-4
num_epochs = 5

trainer = torch.optim.Adam(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss(reduction='none')

In [ ]:
# 开始微调
train_ch13(net,
           train_iter, test_iter,
           loss, trainer,
           num_epochs,
           devices)